In [0]:
%sql
-- Pre-check to inspect the csv file. Do not include it in the job.
/*
select * 
from read_files(
  '/Volumes/job_catalog/default/dataset/orders/',
  format => 'csv'
  );
*/

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, current_timestamp, col

#Ingest data from the orders.csv file and create the bronze_orders table

#Step 1: Read the csv file and create a Spark DataFrame
file_path = "/Volumes/job_catalog/default/dataset/orders/"
csv_schema = StructType([
    StructField('customer_id', IntegerType(), True),
    StructField('customer_name', StringType(), True),
    StructField('product_name', StringType(), True),
    StructField('order_date', DateType(), True),
    StructField('product_category', StringType(), True),
    StructField('total_price', IntegerType(), True)
])

df_orders = (
    spark.read.format("csv")
    .option("header", "true")
    .schema(csv_schema)
    .load(file_path)
    .withColumn("processing_time", current_timestamp())
    .withColumn("file_name", col("_metadata.file_path"))
)

#Step 2: Create a delta table
(df_orders
.write
.mode("overwrite")
.format("delta")
.saveAsTable("job_catalog.default.bronze_orders")
)

In [0]:
# Post-check. Do not include in the job
#spark.read.table("job_catalog.default.bronze_orders").display()